In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### web scraping the table

In [26]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]

In [27]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [28]:
df.shape

(180, 3)

#### Retaining only the cells that have an assigned borough. dropping cells with a borough that is Not assigned.

In [29]:
df['Borough'].unique()

array(['Not assigned', 'North York', 'Downtown Toronto', 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [30]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)

#### More than one neighborhood can exist in one postal code area
#### These two rows will be combined into one row with the neighborhoods separated with a comma 

In [32]:
df1=df.groupby("Postal code").agg(lambda x:','.join(set(x)))

#### If a cell has a borough but a Not assigned neighborhood, then assign neighborhood with same value as the borough.

In [33]:
df1.loc[df1['Neighborhood']=="Not assigned",'Neighborhood']=df1.loc[df1['Neighborhood']=="Not assigned",'Borough']

In [55]:
df1 = df1.reset_index()

In [56]:
df1.shape

(103, 3)

#### the geographical coordinates of each postal code

In [41]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')

In [49]:
geo.columns

Index(['Postal code', 'Latitude', 'Longitude'], dtype='object')

In [45]:
geo.rename(columns={"Postal Code": "Postal code"}, inplace = True)

In [46]:
geo.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [50]:
ndf = pd.merge(df1, geo, on='Postal code', how='inner')

In [51]:
ndf.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
